# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import re
import nltk
from nltk.corpus import (
    stopwords,
    wordnet
)
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV
)
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
print(df.head())
df.info()

   Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
print(df['Unnamed: 0'].is_monotonic)

True


In [4]:
df['Unnamed: 0'].describe()

count    159292.000000
mean      79725.697242
std       46028.837471
min           0.000000
25%       39872.750000
50%       79721.500000
75%      119573.250000
max      159450.000000
Name: Unnamed: 0, dtype: float64

In [5]:
df['toxic'].describe()

# анализируем баланс целевой переменной

count    159292.000000
mean          0.101612
std           0.302139
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: toxic, dtype: float64

In [6]:
df.duplicated().sum()

# анализируем датасет на дубликаты

0

**Выводы**

- исходный датасет содержит три признака без пропусков и дубликатов
- колонка 'Unnamed: 0' похожа на индексы объектов и не несет полезной информации поэтому ее можно удалить
- тексты в колонке 'text' представлены на английском языке с разными регистрами (верхний нижний) поэтому нужно привести к все к нижнему
- баланс целевого признака 'toxic': 90% записей с позитивные (0), 10% записей с токсичные (1) поэтому нужно применить upsampling к обучающей выборке

In [7]:
# удаляем колонку 'Unnamed: 0'

df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
# приводим тексты к нижнему регистру

df['text'] = df['text'].apply(lambda x: x.lower())
df['text'].head()

0    explanation\nwhy the edits made under my usern...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    "\nmore\ni can't make any real suggestions on ...
4    you, sir, are my hero. any chance you remember...
Name: text, dtype: object

In [9]:
def get_wordnet_pos(word):
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [11]:
# создаем функцию обработки текстов

def data_preproc(text):
    
    text =  re.sub('[^A-Za-z0-9]', ' ', text)
    text = text.split()
    text = ' '.join(text)
    
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)] # лемматизация с POS-тегом 
    text = ' '.join(text)

    return text

In [12]:
word_counts = df['text'].apply(lambda x: len(x.split()))
word_counts.describe()

count    159292.000000
mean         67.213545
std          99.133908
min           1.000000
25%          17.000000
50%          36.000000
75%          75.000000
max        1411.000000
Name: text, dtype: float64

In [13]:
# делим данные на признаки X и целевой признак y

X = df.drop(['toxic'], axis=1)
y = df['toxic']

In [14]:
# создаем обучающую, тестовую и валидационную выборки

X_train, X_x, y_train, y_y = train_test_split(
    X, y, test_size=0.4, random_state=12345
)


In [15]:
X_test, X_valid, y_test, y_valid = train_test_split(
    X_x, y_y, test_size=0.5, random_state=12345
)

In [16]:
X_train = X_train['text'].apply(lambda text: data_preproc(text))

In [17]:
X_test = X_test['text'].apply(lambda text: data_preproc(text))

In [18]:
X_valid = X_valid['text'].apply(lambda text: data_preproc(text))

### Векторизация

**Мешок слов**

In [19]:
# создаем переменные
corpus_train = X_train.values
corpus_valid = X_valid.values
corpus_test = X_test.values

In [20]:
# создаем мешок слов
count_vect_bow = CountVectorizer(dtype='float64', stop_words=stop_words) 
bow_train = count_vect_bow.fit_transform(corpus_train)
bow_valid = count_vect_bow.transform(corpus_valid)
bow_train.shape

(95575, 120004)

**tf_idf**

In [21]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)
tf_idf_train.shape

(95575, 120004)

**n-граммы**

count_vect = CountVectorizer(ngram_range=(3, 3), stop_words=stop_words)
n_gramm_train = count_vect.fit_transform(corpus_train)
n_gramm_train.shape

In [22]:
model = LogisticRegression(random_state = 12345)

In [23]:
for i in [bow_valid, tf_idf_valid]:
    f1 = cross_val_score(model, i, y_valid, cv=5, scoring='f1')  
    print(f1.mean())

0.7160503630693594
0.5916030620202536


**Выводы**

- произвели векторизацию текстов тремя способами (мешок слов, tf_idf, n-граммы)
- на модели LR проверили на какой векторизации показывает лучший результат по метрики F1
- n-граммы лучший показатель метрики f1, но использовать будем мешок слов т.к. меньше признаков на порядок

## Обучение

In [24]:
# векторизируем тестовую выборку
bow_test = count_vect_bow.transform(corpus_test)
bow_test.shape

(31858, 120004)

### LogisticRegression

In [71]:
lr_model = LogisticRegression(random_state = 12345, solver = 'lbfgs', class_weight='balanced', C=3)

In [72]:
lr_model.fit(bow_train, y_train)

LogisticRegression(C=3, class_weight='balanced', random_state=12345)

f1_lr = cross_val_score(lr_model, bow_valid, y_valid, cv=5, scoring='f1')
f1_lr.mean()

### LGBMClassifier 

lgbm_model = LGBMClassifier(random_state = 12345, max_depth = 4, num_leaves = 10, learning_rate = 0.4)

#scores = cross_val_score(lgbm_model, bow_train, y_train, cv=5, scoring='f1_weighted')
#print('F1  LGBM:', scores.mean())
lgbm_model.fit(bow_train, y_train)
predicted_train = lgbm_model.predict(bow_train)
f1_lg = f1_score(y_train, predicted_train)
f1_lg

### Проверка лучшей модели (LR) на тестовой выборке

In [73]:
predicted_test = lr_model.predict(bow_test)

In [75]:
print(f1_score(y_test, predicted_test).round(2))


0.75


 **Выводы**

- была обучена а модель LogisticRegression(random_state = 12345, solver = 'lbfgs', class_weight='balanced', C=3) 
- показатель f1 на тестовой метрики 0.75 

## Выводы

- анализ показал что нужно использовать модель LR
- метод векторизации лучшего всего подходит n-граммы (мы использовали n=3), но использовали мешок слов т.к. меньше признаков на порядок
- на тестовой выборке лучшая модель LR показала f1= что удовлитворяет условию поставленной задачи